In [1]:
# Phishing Email Detection Using TF-IDF + SVM and Fine-tuned DistilBERT

# 1. Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# 2. Load Dataset
# Dataset: phishing_email.csv from Kaggle

# Important: Make sure the CSV file has columns 'body' and 'label'
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/phishing_email.csv')
data.head()

In [ ]:
# 3. Preprocessing
# Basic text cleaning
data['body'] = data['body'].str.lower()

In [ ]:
# 4. Train/Test Split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['body'].tolist(), data['label'].tolist(), test_size=0.2, random_state=42)


In [ ]:
# 5. Baseline Model: TF-IDF + SVM
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

baseline_model = SVC(kernel='linear', probability=True)
baseline_model.fit(X_train, train_labels)

baseline_preds = baseline_model.predict(X_test)
print("\n=== Baseline Model Evaluation (TF-IDF + SVM) ===")
print(classification_report(test_labels, baseline_preds)) 

In [ ]:
# 6. DistilBERT Fine-tuning
# Tokenization

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Dataset Class
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)

# Load Model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune DistilBERT
trainer.train()

In [ ]:
# 7. Evaluate DistilBERT Model
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

print("\n=== DistilBERT Model Evaluation ===")
print(classification_report(test_labels, preds))

In [ ]:
# 8. Confusion Matrix for DistilBERT
conf_matrix = confusion_matrix(test_labels, preds)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix: DistilBERT Model')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# 9. Save Models (Optional)
torch.save(model.state_dict(), 'distilbert_phishing_model.pt')
joblib.dump(baseline_model, 'baseline_svm_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')